In [57]:
print("OK")

OK


# Q&A over the Code Base to Understand How it Works

In [58]:
from git import Repo
import os

from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [59]:
%pwd

'/home/fahad/Documents/Projects/SPL Excessories/SPL3/others/Test v2'

In [60]:
# !mkdir test_repo

In [61]:
repo_path = "test_repo/pytagfix-master/"
# repo = Repo.clone_from("https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI", to_path=repo_path)

## Load Project (Documents)

In [62]:
loader = GenericLoader.from_filesystem(repo_path,
                                       glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [63]:
documents = loader.load()

In [64]:
documents

[Document(metadata={'source': 'test_repo/pytagfix-master/test.py', 'language': <Language.PYTHON: 'python'>}, page_content='# custom type\nclass Student:\n    def __init__(self, name: str, age: int, school: str):\n        self.name = name\n        self.age = age\n        self.school = school\n\n\ndef print_student_details(student: Student) -> None:\n    print(student.name, student.age)\n\n\n# newtype\nfrom typing import NewType\n\nStudentID = NewType("StudentID", int)\nCourseID = NewType("CourseID", int)\n\n\ndef get_student(student_id: StudentID) -> None:\n    pass\n\n\ndef enroll_in_course(student_id: StudentID, course_id: CourseID) -> None:\n    pass\n\n\n# generics\nfrom typing import List, TypeVar\n\nT = TypeVar("T")\n\n\ndef first_element(lst: List[T]) -> T:\n    return lst[0]\n\n\nfrom typing import Generic, TypeVar\n\nT = TypeVar("T")\n\n\nclass Box(Generic[T]):\n    def __init__(self, value: T):\n        self.value = value\n\n    def get(self) -> T:\n        return self.value\n

In [65]:
len(documents[0].page_content)

1946

In [66]:
documents[0]

Document(metadata={'source': 'test_repo/pytagfix-master/test.py', 'language': <Language.PYTHON: 'python'>}, page_content='# custom type\nclass Student:\n    def __init__(self, name: str, age: int, school: str):\n        self.name = name\n        self.age = age\n        self.school = school\n\n\ndef print_student_details(student: Student) -> None:\n    print(student.name, student.age)\n\n\n# newtype\nfrom typing import NewType\n\nStudentID = NewType("StudentID", int)\nCourseID = NewType("CourseID", int)\n\n\ndef get_student(student_id: StudentID) -> None:\n    pass\n\n\ndef enroll_in_course(student_id: StudentID, course_id: CourseID) -> None:\n    pass\n\n\n# generics\nfrom typing import List, TypeVar\n\nT = TypeVar("T")\n\n\ndef first_element(lst: List[T]) -> T:\n    return lst[0]\n\n\nfrom typing import Generic, TypeVar\n\nT = TypeVar("T")\n\n\nclass Box(Generic[T]):\n    def __init__(self, value: T):\n        self.value = value\n\n    def get(self) -> T:\n        return self.value\n\

## Split Document to store in Vector DB

In [67]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [68]:
texts = documents_splitter.split_documents(documents)

In [69]:
len(texts[0].page_content)

13

In [70]:
len(texts)

40

## Generate Embedding

In [71]:
from dotenv import load_dotenv
load_dotenv() 

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [72]:
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY

In [73]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings=OpenAIEmbeddings(disallowed_special=())
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [74]:
from langchain_chroma import Chroma

vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./db')

In [75]:
# vectordb.persist()

## Query Vector DB & Create QA Agent

In [76]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.5, max_tokens=4018, google_api_key=GOOGLE_API_KEY)

In [77]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", output_key='answer', return_messages=True)

In [78]:
qa = ConversationalRetrievalChain.from_llm(
    llm, 
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 8}), 
    memory=memory, 
    return_source_documents=True, 
    output_key="answer"  
)

In [79]:
question = ''' 
what are the python scripts in this project? mention the names.
'''

In [80]:
result = qa(question)
print(result['answer'])

The Python scripts mentioned in the provided code snippets are:

*   `LoginProtocol.py`
*   `run_tests.py`
*   `main.py` (imported by `run_tests.py`)
*   `config.py` (imported as `imported_config`)
*   `Server.py` (within the `tibiaproxy` directory)
*   `util.py` (within the `tibiaproxy` directory, containing the `log` function)
*   A script that utilizes `prettytitle` (the exact filename isn't provided, but the import suggests its existence).






## Ask your LLM

In [81]:
# source_code = 'def walk_metas(app):\n # Iterates from deepest to shallowest meta-apps meta_list = [app]  # shallowest to deepest meta = app while (meta := meta._meta) and meta.default_command:'

'''ArithmeticError
message =
'lang is declared to have type `str` but is used as type `None`.'
rule_id =
'Incompatible variable type [9]'
source_code =
'def pretty_string(s: str, lang: str = None) -> str:  ##! RESOLVE this error\n """Return correctly titled string""" newwords: list = [] newsent: bool = True  # Lang detect. FIXME: Primitive for now if lang is None:'
warning_line =
'def pretty_string(s: str, lang: str = None) -> str:  ##! RESOLVE this error'
Line: 110, Col: 27
prettytitle.py
'''  
 

 
question = f'''
You are a Python type hint error resolver. Your task is to fix type-related errors in Python code by generating the correct type hints and patches.

Error Details:
- Error Message: lang is declared to have type `str` but is used as type `None`.
- PyRe Error Type: Incompatible variable type [9]
- File: prettytitle.py
- Line: 102

Code Snippet:       return pw_rus(s, newsent)

Note: Before solving, find the function and related statement with given Code Snippet and make changes to them for final resolution.

Provide:
1. The corrected code with appropriate type hints.
2. A brief explanation of the changes made.
'''

In [82]:
result = qa({"question": question})
print(result['answer'])

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


The error arises because the `lang` parameter in `pretty_string` is typed as `str` but can implicitly become `None` due to the conditional logic at the beginning of the function.  The `pretty_word` function (which calls `pw_rus` and `pw_eng`) also expects `lang` to be a string.  Therefore, we need to allow `lang` to be optionally `None` in both functions and handle the case where it is `None` appropriately.

```python
from typing import Optional

CYRILLIC_LETTERS = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
CONSONANTS = "bcdfghjklmnpqrstvwxz"
PLUSES = "ьъ"

# ... other code (pw_eng, pw_rus implementations etc.)

def pretty_string(s: str, lang: Optional[str] = None) -> str:
    """Return correctly titled string"""
    newwords: list = []
    newsent: bool = True

    # Lang detect. FIXME: Primitive for now
    if lang is None:
        if any(x in s for x in CYRILLIC_LETTERS):
            lang = "rus"
        else:
            lang = "eng"

    for word in s.split():
        newwords.append(pre